# R1: **Single product and stochastic environment**

ADVERTISER: Company
----

single ptoduct

In [ ]:
class Company_R1:
    def __init__(self, T, M, P):
        self.T = T  # Number of rounds
        self.N = 1  # one product 
        self.M = M  # Set of possible prices (LIST discrete set)
        self.P = P  # Production capacity budget (INT)

        self.products = P   # ONE product = all capacity in one product
        self.prices = None  # Prices for the product

    def sell_products(self):
        # sell one product
        if self.products > 0:
            print(f"Products sold, Remained {self.products}")
            self.products -= 1
        else:
            print("No products to sell")

    def set_prices(self):   
        # implement the logic to set prices UCB1
        pass

### BUYER: User

In [ ]:
class User:
    def __init__(self, valuation):
        self.valuation = valuation  # Valuation of the product

    def buy_product(self, price, quantity):
        if self.valuation >= price and quantity > 0:
            print(f"Product bought at price {price}")
            return True

### INTERACTION